# Thermal model recognizing objects without specific class 

`./FLIR/dataset_final` has all the FLIR thermal dataset. I'll now add our dataset to it

In [3]:
import os
import shutil
import random
from math import floor

def create_directory_structure(base_dir):
    for split in ['train', 'test', 'val']:
        images_dir = os.path.join(base_dir, split, 'images')
        labels_dir = os.path.join(base_dir, split, 'labels')
        os.makedirs(images_dir, exist_ok=True)
        os.makedirs(labels_dir, exist_ok=True)

def split_data(images_dir, labels_dir, output_dir, train_ratio=0.7, test_ratio=0.2, val_ratio=0.1):
    images = [img for img in os.listdir(images_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]
    labels = [lbl for lbl in os.listdir(labels_dir) if lbl.endswith('.txt')]
    
    images.sort()
    labels.sort()
    
    assert len(images) == len(labels), "The number of images and labels must be the same"
    
    paired_files = list(zip(images, labels))
    random.shuffle(paired_files)
    
    total_count = len(paired_files)
    train_count = floor(total_count * train_ratio)
    test_count = floor(total_count * test_ratio)
    val_count = total_count - train_count - test_count
    
    train_files = paired_files[:train_count]
    test_files = paired_files[train_count:train_count + test_count]
    val_files = paired_files[train_count + test_count:]
    
    def copy_files(file_pairs, split_name):
        for img_file, lbl_file in file_pairs:
            shutil.copy(os.path.join(images_dir, img_file), os.path.join(output_dir, split_name, 'images', img_file))
            shutil.copy(os.path.join(labels_dir, lbl_file), os.path.join(output_dir, split_name, 'labels', lbl_file))
    
    copy_files(train_files, 'train')
    copy_files(test_files, 'test')
    copy_files(val_files, 'val')

Creat

In [10]:
images_dir = "/data1/jingxia3/SURA/BimodalVideo/thermal_frames_cropped"
labels_dir = "/data1/jingxia3/SURA/BimodalVideo/labels"
output_dir = "/data1/jingxia3/SURA/BimodalVideo/splitted"

create_directory_structure(output_dir)
split_data(images_dir, labels_dir, output_dir)

In [12]:
!cp -rf /data1/jingxia3/SURA/FLIR/final_dataset /data1/jingxia3/SURA/BimodalVideo/dataset_merged

!cp /data1/jingxia3/SURA/BimodalVideo/splitted/train/images/* /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/images/
!cp /data1/jingxia3/SURA/BimodalVideo/splitted/train/labels/* /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/labels/

!cp /data1/jingxia3/SURA/BimodalVideo/splitted/test/images/* /data1/jingxia3/SURA/BimodalVideo/dataset_merged/test/images/
!cp /data1/jingxia3/SURA/BimodalVideo/splitted/test/labels/* /data1/jingxia3/SURA/BimodalVideo/dataset_merged/test/labels/

!cp /data1/jingxia3/SURA/BimodalVideo/splitted/val/images/* /data1/jingxia3/SURA/BimodalVideo/dataset_merged/val/images/
!cp /data1/jingxia3/SURA/BimodalVideo/splitted/val/labels/* /data1/jingxia3/SURA/BimodalVideo/dataset_merged/val/labels/

Then I removed all labels

In [13]:
def rename_labels(new_label_dir):
    new_label_files = [f for f in os.listdir(new_label_dir) if f.endswith('.txt')]

    for new_label_file in new_label_files:
        file_path = new_label_dir + new_label_file
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        modified_lines = []
        for line in lines:
            columns = line.split()
            columns[0] = '0'  # Change the first column to 0
            modified_line = ' '.join(columns)
            modified_lines.append(modified_line)
        
        with open(file_path, 'w') as file:
            for modified_line in modified_lines:
                file.write(modified_line + '\n')
    
    print(f"Done with {len(new_label_files)} files!")

In [14]:
rename_labels('/data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/labels/')
rename_labels('/data1/jingxia3/SURA/BimodalVideo/dataset_merged/test/labels/')
rename_labels('/data1/jingxia3/SURA/BimodalVideo/dataset_merged/val/labels/')

Done with 11375 files!
Done with 3930 files!
Done with 1235 files!


Now the training after changing `data.yaml`!

In [16]:
!cp /data1/jingxia3/SURA/FLIR/dataset_binary/data.yaml /data1/jingxia3/SURA/BimodalVideo/dataset_merged/

In [27]:
from ultralytics import YOLO
backbone = YOLO("yolov8n.yaml")

In [28]:
dataset = "/data1/jingxia3/SURA/BimodalVideo/dataset_merged/data.yaml"
train_res = backbone.train(data=dataset)

New https://pypi.org/project/ultralytics/8.2.50 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/data1/jingxia3/SURA/BimodalVideo/dataset_merged/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train24, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

train: Scanning /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/labels.ca

train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/images/video-5v4MPkCMGEKPfroxc-frame-000285-M8qDkmPHxWQajYdEK.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/images/video-BjSfmxLQqCGjg8tya-frame-004259-dquYSgZ4rzqYqhtbc.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/images/video-JFiGa9oAAN2queuLt-frame-007561-jKiKkJJY9yjxZQjDi.jpg: 1 duplicate labels removed



val: Scanning /data1/jingxia3/SURA/BimodalVideo/dataset_merged/val/labels.cache.

val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-005290-Rkhr9LNNEftZzJ98w.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-008818-M29zA3RYL5ud23Cp9.jpg: 5 duplicate labels removed


...

    100/100      4.63G      1.161     0.6639      1.001        169        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all       1235      16901      0.811      0.618      0.722      0.452



100 epochs completed in 1.596 hours.
Optimizer stripped from runs/detect/train24/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train24/weights/best.pt, 6.2MB

Validating runs/detect/train24/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1235      16901      0.811      0.618      0.722      0.452
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train24


In [29]:
backbone.export()

Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CPU (Intel Xeon Gold 6342 2.80GHz)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients

PyTorch: starting from 'runs/detect/train24/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

TorchScript: starting export with torch 2.3.0...
TorchScript: export success ✅ 0.9s, saved as 'runs/detect/train24/weights/best.torchscript' (11.9 MB)

Export complete (2.3s)
Results saved to /data1/jingxia3/SURA/runs/detect/train24/weights
Predict:         yolo predict task=detect model=runs/detect/train24/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train24/weights/best.torchscript imgsz=640 data=/data1/jingxia3/SURA/BimodalVideo/dataset_merged/data.yaml  
Visualize:       https://netron.app


'runs/detect/train24/weights/best.torchscript'

## What about for medium model?

In [31]:
medium_backbone = YOLO("yolov8m.yaml")
medium_res = medium_backbone.train(data=dataset)

New https://pypi.org/project/ultralytics/8.2.51 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/data1/jingxia3/SURA/BimodalVideo/dataset_merged/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

train: Scanning /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/labels.ca

train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/images/video-5v4MPkCMGEKPfroxc-frame-000285-M8qDkmPHxWQajYdEK.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/images/video-BjSfmxLQqCGjg8tya-frame-004259-dquYSgZ4rzqYqhtbc.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/images/video-JFiGa9oAAN2queuLt-frame-007561-jKiKkJJY9yjxZQjDi.jpg: 1 duplicate labels removed



val: Scanning /data1/jingxia3/SURA/BimodalVideo/dataset_merged/val/labels.cache.

val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-005290-Rkhr9LNNEftZzJ98w.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-008818-M29zA3RYL5ud23Cp9.jpg: 5 duplicate labels removed


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      7.48G     0.9141     0.4664     0.9159        169        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1235      16901      0.844      0.694      0.792      0.527

100 epochs completed in 2.683 hours.
Optimizer stripped from runs/detect/train25/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train25/weights/best.pt, 52.0MB

Validating runs/detect/train25/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8m summary (fused): 218 layers, 25840339 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1235      16901      0.846      0.695      0.793      0.527
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train25


## Small?

In [32]:
small_backbone = YOLO("yolov8s.yaml")
medium_res = small_backbone.train(data=dataset)

New https://pypi.org/project/ultralytics/8.2.51 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/data1/jingxia3/SURA/BimodalVideo/dataset_merged/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train26, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

train: Scanning /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/labels.ca

train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/images/video-5v4MPkCMGEKPfroxc-frame-000285-M8qDkmPHxWQajYdEK.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/images/video-BjSfmxLQqCGjg8tya-frame-004259-dquYSgZ4rzqYqhtbc.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/train/images/video-JFiGa9oAAN2queuLt-frame-007561-jKiKkJJY9yjxZQjDi.jpg: 1 duplicate labels removed



val: Scanning /data1/jingxia3/SURA/BimodalVideo/dataset_merged/val/labels.cache.

val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-005290-Rkhr9LNNEftZzJ98w.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/dataset_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-008818-M29zA3RYL5ud23Cp9.jpg: 5 duplicate labels removed


...

    100/100      5.08G      1.023     0.5489     0.9605        169        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all       1235      16901      0.825       0.67      0.768      0.498



100 epochs completed in 1.734 hours.
Optimizer stripped from runs/detect/train26/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train26/weights/best.pt, 22.5MB

Validating runs/detect/train26/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8s summary (fused): 168 layers, 11125971 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1235      16901      0.831      0.667      0.768      0.498
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train26


## Different colors

HSV

In [33]:
images_dir = "/data1/jingxia3/SURA/BimodalVideo/output_hsv"  # Replace with your images folder path
labels_dir = "/data1/jingxia3/SURA/BimodalVideo/labels"  # Replace with your labels folder path
output_dir = "/data1/jingxia3/SURA/BimodalVideo/splitted"  # Replace with your desired output folder path

create_directory_structure(output_dir)
split_data(images_dir, labels_dir, output_dir)

In [36]:
!cp -rf /data1/jingxia3/SURA/BimodalVideo/dataset_merged /data1/jingxia3/SURA/BimodalVideo/hsv_merged

!cp /data1/jingxia3/SURA/BimodalVideo/splitted/train/images/* /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/images/
!cp /data1/jingxia3/SURA/BimodalVideo/splitted/train/labels/* /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/labels/

!cp /data1/jingxia3/SURA/BimodalVideo/splitted/test/images/* /data1/jingxia3/SURA/BimodalVideo/hsv_merged/test/images/
!cp /data1/jingxia3/SURA/BimodalVideo/splitted/test/labels/* /data1/jingxia3/SURA/BimodalVideo/hsv_merged/test/labels/

!cp /data1/jingxia3/SURA/BimodalVideo/splitted/val/images/* /data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/images/
!cp /data1/jingxia3/SURA/BimodalVideo/splitted/val/labels/* /data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/labels/

In [37]:
rename_labels('/data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/labels/')
rename_labels('/data1/jingxia3/SURA/BimodalVideo/hsv_merged/test/labels/')
rename_labels('/data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/labels/')

Done with 11375 files!
Done with 3930 files!
Done with 1235 files!


In [41]:
!cp /data1/jingxia3/SURA/FLIR/images_thermal_train/output_hsv/* /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/images/
!cp /data1/jingxia3/SURA/FLIR/video_thermal_test/output_hsv/* /data1/jingxia3/SURA/BimodalVideo/hsv_merged/test/images/
!cp /data1/jingxia3/SURA/FLIR/images_thermal_val/output_hsv/* /data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/images/

Hot

In [38]:
images_dir = "/data1/jingxia3/SURA/BimodalVideo/output_hot"  # Replace with your images folder path
labels_dir = "/data1/jingxia3/SURA/BimodalVideo/labels"  # Replace with your labels folder path
output_dir = "/data1/jingxia3/SURA/BimodalVideo/splitted"  # Replace with your desired output folder path

create_directory_structure(output_dir)
split_data(images_dir, labels_dir, output_dir)

In [39]:
!cp -rf /data1/jingxia3/SURA/BimodalVideo/dataset_merged /data1/jingxia3/SURA/BimodalVideo/hot_merged

!cp /data1/jingxia3/SURA/BimodalVideo/splitted/train/images/* /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/images/
!cp /data1/jingxia3/SURA/BimodalVideo/splitted/train/labels/* /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/labels/

!cp /data1/jingxia3/SURA/BimodalVideo/splitted/test/images/* /data1/jingxia3/SURA/BimodalVideo/hot_merged/test/images/
!cp /data1/jingxia3/SURA/BimodalVideo/splitted/test/labels/* /data1/jingxia3/SURA/BimodalVideo/hot_merged/test/labels/

!cp /data1/jingxia3/SURA/BimodalVideo/splitted/val/images/* /data1/jingxia3/SURA/BimodalVideo/hot_merged/val/images/
!cp /data1/jingxia3/SURA/BimodalVideo/splitted/val/labels/* /data1/jingxia3/SURA/BimodalVideo/hot_merged/val/labels/

In [40]:
rename_labels('/data1/jingxia3/SURA/BimodalVideo/hot_merged/train/labels/')
rename_labels('/data1/jingxia3/SURA/BimodalVideo/hot_merged/test/labels/')
rename_labels('/data1/jingxia3/SURA/BimodalVideo/hot_merged/val/labels/')

Done with 11578 files!
Done with 4082 files!
Done with 1316 files!


In [42]:
!cp /data1/jingxia3/SURA/FLIR/images_thermal_train/output_hot/* /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/images/
!cp /data1/jingxia3/SURA/FLIR/video_thermal_test/output_hot/* /data1/jingxia3/SURA/BimodalVideo/hot_merged/test/images/
!cp /data1/jingxia3/SURA/FLIR/images_thermal_val/output_hot/* /data1/jingxia3/SURA/BimodalVideo/hot_merged/val/images/

Inferno

In [43]:
images_dir = "/data1/jingxia3/SURA/BimodalVideo/output_inferno"  # Replace with your images folder path
labels_dir = "/data1/jingxia3/SURA/BimodalVideo/labels"  # Replace with your labels folder path
output_dir = "/data1/jingxia3/SURA/BimodalVideo/splitted"  # Replace with your desired output folder path

create_directory_structure(output_dir)
split_data(images_dir, labels_dir, output_dir)

In [44]:
!cp -rf /data1/jingxia3/SURA/BimodalVideo/dataset_merged /data1/jingxia3/SURA/BimodalVideo/inferno_merged

!cp /data1/jingxia3/SURA/BimodalVideo/splitted/train/images/* /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/images/
!cp /data1/jingxia3/SURA/BimodalVideo/splitted/train/labels/* /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/labels/

!cp /data1/jingxia3/SURA/BimodalVideo/splitted/test/images/* /data1/jingxia3/SURA/BimodalVideo/inferno_merged/test/images/
!cp /data1/jingxia3/SURA/BimodalVideo/splitted/test/labels/* /data1/jingxia3/SURA/BimodalVideo/inferno_merged/test/labels/

!cp /data1/jingxia3/SURA/BimodalVideo/splitted/val/images/* /data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/images/
!cp /data1/jingxia3/SURA/BimodalVideo/splitted/val/labels/* /data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/labels/

In [45]:
rename_labels('/data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/labels/')
rename_labels('/data1/jingxia3/SURA/BimodalVideo/inferno_merged/test/labels/')
rename_labels('/data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/labels/')

Done with 11631 files!
Done with 4193 files!
Done with 1394 files!


In [46]:
!cp /data1/jingxia3/SURA/FLIR/images_thermal_train/output_inferno/* /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/images/
!cp /data1/jingxia3/SURA/FLIR/video_thermal_test/output_inferno/* /data1/jingxia3/SURA/BimodalVideo/inferno_merged/test/images/
!cp /data1/jingxia3/SURA/FLIR/images_thermal_val/output_inferno/* /data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/images/

## Training with mapping

In [1]:
from ultralytics import YOLO
hsv_backbone = YOLO("yolov8n.yaml")
hsv_res = hsv_backbone.train(data='/data1/jingxia3/SURA/BimodalVideo/hsv_merged/data.yaml')

New https://pypi.org/project/ultralytics/8.2.54 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/data1/jingxia3/SURA/BimodalVideo/hsv_merged/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train36, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

train: Scanning /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/labels.cache... 11375 images, 1047 backgrounds, 0 corrupt: 100%|██████████| 11924/11924 [00:00<?, ?it/s]

train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/images/video-5v4MPkCMGEKPfroxc-frame-000285-M8qDkmPHxWQajYdEK.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/images/video-BjSfmxLQqCGjg8tya-frame-004259-dquYSgZ4rzqYqhtbc.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/images/video-JFiGa9oAAN2queuLt-frame-007561-jKiKkJJY9yjxZQjDi.jpg: 1 duplicate labels removed



val: Scanning /data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/labels.cache... 1235 images, 51 backgrounds, 0 corrupt: 100%|██████████| 1235/1235 [00:00<?, ?it/s]

val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-005290-Rkhr9LNNEftZzJ98w.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-008818-M29zA3RYL5ud23Cp9.jpg: 5 duplicate labels removed


...

                   all       1235      16901      0.778      0.558      0.653       0.39

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      2.65G      1.243     0.7519      1.022         59        640: 100%|██████████| 746/746 [00:50<00:00, 14.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [00:03<00:00, 11.62it/s]


                   all       1235      16901      0.778      0.558      0.653       0.39

100 epochs completed in 1.507 hours.
Optimizer stripped from runs/detect/train36/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train36/weights/best.pt, 6.2MB

Validating runs/detect/train36/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [00:04<00:00,  8.12it/s]


                   all       1235      16901      0.778      0.558      0.653       0.39
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train36


In [2]:
hot_backbone = YOLO("yolov8n.yaml")
hot_res = hot_backbone.train(data='/data1/jingxia3/SURA/BimodalVideo/hot_merged/data.yaml')

New https://pypi.org/project/ultralytics/8.2.54 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/data1/jingxia3/SURA/BimodalVideo/hot_merged/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train37, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

train: Scanning /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/labels.cache... 11578 images, 1141 backgrounds, 0 corrupt: 100%|██████████| 12127/12127 [00:00<?, ?it/s]

train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/images/video-5v4MPkCMGEKPfroxc-frame-000285-M8qDkmPHxWQajYdEK.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/images/video-BjSfmxLQqCGjg8tya-frame-004259-dquYSgZ4rzqYqhtbc.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/images/video-JFiGa9oAAN2queuLt-frame-007561-jKiKkJJY9yjxZQjDi.jpg: 1 duplicate labels removed



val: Scanning /data1/jingxia3/SURA/BimodalVideo/hot_merged/val/labels.cache... 1316 images, 80 backgrounds, 0 corrupt: 100%|██████████| 1316/1316 [00:00<?, ?it/s]

val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-005290-Rkhr9LNNEftZzJ98w.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-008818-M29zA3RYL5ud23Cp9.jpg: 5 duplicate labels removed


...

                   all       1316      17073      0.786      0.571      0.665      0.399

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      2.86G       1.26     0.7722      1.022        183        640: 100%|██████████| 758/758 [00:48<00:00, 15.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:03<00:00, 12.05it/s]


                   all       1316      17073      0.787      0.571      0.665      0.399

100 epochs completed in 1.536 hours.
Optimizer stripped from runs/detect/train37/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train37/weights/best.pt, 6.2MB

Validating runs/detect/train37/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:05<00:00,  8.31it/s]


                   all       1316      17073      0.786      0.571      0.665      0.399
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train37


In [3]:
inferno_backbone = YOLO("yolov8n.yaml")
inferno_res = inferno_backbone.train(data='/data1/jingxia3/SURA/BimodalVideo/inferno_merged/data.yaml')

New https://pypi.org/project/ultralytics/8.2.54 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/data1/jingxia3/SURA/BimodalVideo/inferno_merged/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train38, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

train: Scanning /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/labels.cache... 11631 images, 1159 backgrounds, 0 corrupt: 100%|██████████| 12180/12180 [00:00<?, ?it/s]

train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/images/video-5v4MPkCMGEKPfroxc-frame-000285-M8qDkmPHxWQajYdEK.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/images/video-BjSfmxLQqCGjg8tya-frame-004259-dquYSgZ4rzqYqhtbc.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/images/video-JFiGa9oAAN2queuLt-frame-007561-jKiKkJJY9yjxZQjDi.jpg: 1 duplicate labels removed



val: Scanning /data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/labels.cache... 1394 images, 112 backgrounds, 0 corrupt: 100%|██████████| 1394/1394 [00:00<?, ?it/s]

val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-005290-Rkhr9LNNEftZzJ98w.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-008818-M29zA3RYL5ud23Cp9.jpg: 5 duplicate labels removed


...

                   all       1394      17223       0.78      0.574      0.668      0.401

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      2.93G      1.243     0.7588      1.015         27        640: 100%|██████████| 762/762 [00:51<00:00, 14.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:03<00:00, 11.89it/s]


                   all       1394      17223       0.78      0.574      0.668      0.401

100 epochs completed in 1.590 hours.
Optimizer stripped from runs/detect/train38/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train38/weights/best.pt, 6.2MB

Validating runs/detect/train38/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:05<00:00,  8.54it/s]


                   all       1394      17223      0.779      0.575      0.668      0.401
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train38


## Small?

In [4]:
hsv_backbone_s = YOLO("yolov8s.yaml")
hsv_res_s = hsv_backbone_s.train(data='/data1/jingxia3/SURA/BimodalVideo/hsv_merged/data.yaml')

New https://pypi.org/project/ultralytics/8.2.54 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/data1/jingxia3/SURA/BimodalVideo/hsv_merged/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train39, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

train: Scanning /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/labels.cache... 11375 images, 1047 backgrounds, 0 corrupt: 100%|██████████| 11924/11924 [00:00<?, ?it/s]

train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/images/video-5v4MPkCMGEKPfroxc-frame-000285-M8qDkmPHxWQajYdEK.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/images/video-BjSfmxLQqCGjg8tya-frame-004259-dquYSgZ4rzqYqhtbc.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/images/video-JFiGa9oAAN2queuLt-frame-007561-jKiKkJJY9yjxZQjDi.jpg: 1 duplicate labels removed



val: Scanning /data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/labels.cache... 1235 images, 51 backgrounds, 0 corrupt: 100%|██████████| 1235/1235 [00:00<?, ?it/s]

val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-005290-Rkhr9LNNEftZzJ98w.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-008818-M29zA3RYL5ud23Cp9.jpg: 5 duplicate labels removed


...

    100/100      4.25G      1.087     0.6311     0.9702         59        640: 100%|██████████| 746/746 [00:59<00:00, 12.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [00:03<00:00, 11.03it/s]

                   all       1235      16901      0.807      0.616       0.71      0.438



100 epochs completed in 1.793 hours.
Optimizer stripped from runs/detect/train39/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train39/weights/best.pt, 22.5MB

Validating runs/detect/train39/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8s summary (fused): 168 layers, 11125971 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [00:05<00:00,  7.69it/s]


                   all       1235      16901      0.805      0.618      0.711      0.439
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train39


In [6]:
hot_backbone_s = YOLO("yolov8s.yaml")
hot_res_s = hot_backbone_s.train(data='/data1/jingxia3/SURA/BimodalVideo/hot_merged/data.yaml')

New https://pypi.org/project/ultralytics/8.2.54 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/data1/jingxia3/SURA/BimodalVideo/hot_merged/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train41, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

train: Scanning /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/labels.cache... 11578 images, 1141 backgrounds, 0 corrupt: 100%|██████████| 12127/12127 [00:00<?, ?it/s]

train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/images/video-5v4MPkCMGEKPfroxc-frame-000285-M8qDkmPHxWQajYdEK.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/images/video-BjSfmxLQqCGjg8tya-frame-004259-dquYSgZ4rzqYqhtbc.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/images/video-JFiGa9oAAN2queuLt-frame-007561-jKiKkJJY9yjxZQjDi.jpg: 1 duplicate labels removed



val: Scanning /data1/jingxia3/SURA/BimodalVideo/hot_merged/val/labels.cache... 1316 images, 80 backgrounds, 0 corrupt: 100%|██████████| 1316/1316 [00:00<?, ?it/s]

val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-005290-Rkhr9LNNEftZzJ98w.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-008818-M29zA3RYL5ud23Cp9.jpg: 5 duplicate labels removed


...

                   all       1316      17073      0.815      0.617      0.717      0.447

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      4.96G      1.115     0.6543     0.9785        183        640: 100%|██████████| 758/758 [01:01<00:00, 12.34it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:04<00:00,  9.46it/s]


                   all       1316      17073      0.814      0.617      0.717      0.447

100 epochs completed in 1.885 hours.
Optimizer stripped from runs/detect/train41/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train41/weights/best.pt, 22.5MB

Validating runs/detect/train41/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8s summary (fused): 168 layers, 11125971 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:06<00:00,  6.22it/s]


                   all       1316      17073      0.813      0.617      0.717      0.447
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train41


In [7]:
inferno_backbone_s = YOLO("yolov8s.yaml")
inferno_res_s = inferno_backbone_s.train(data='/data1/jingxia3/SURA/BimodalVideo/inferno_merged/data.yaml')

New https://pypi.org/project/ultralytics/8.2.55 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/data1/jingxia3/SURA/BimodalVideo/inferno_merged/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train42, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

train: Scanning /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/labels.cache... 11631 images, 1159 backgrounds, 0 corrupt: 100%|██████████| 12180/12180 [00:00<?, ?it/s]

train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/images/video-5v4MPkCMGEKPfroxc-frame-000285-M8qDkmPHxWQajYdEK.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/images/video-BjSfmxLQqCGjg8tya-frame-004259-dquYSgZ4rzqYqhtbc.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/images/video-JFiGa9oAAN2queuLt-frame-007561-jKiKkJJY9yjxZQjDi.jpg: 1 duplicate labels removed



val: Scanning /data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/labels.cache... 1394 images, 112 backgrounds, 0 corrupt: 100%|██████████| 1394/1394 [00:00<?, ?it/s]

val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-005290-Rkhr9LNNEftZzJ98w.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-008818-M29zA3RYL5ud23Cp9.jpg: 5 duplicate labels removed


...

                   all       1394      17223      0.806      0.631      0.725      0.453

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      5.14G      1.098     0.6397     0.9748         27        640: 100%|██████████| 762/762 [01:02<00:00, 12.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:04<00:00, 10.36it/s]


                   all       1394      17223      0.807      0.631      0.725      0.453

100 epochs completed in 2.619 hours.
Optimizer stripped from runs/detect/train42/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train42/weights/best.pt, 22.5MB

Validating runs/detect/train42/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8s summary (fused): 168 layers, 11125971 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:05<00:00,  7.50it/s]


                   all       1394      17223      0.805      0.631      0.725      0.453
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train42


## Medium?

In [8]:
hsv_backbone_m = YOLO("yolov8m.yaml")
hsv_res_m = hsv_backbone_m.train(data='/data1/jingxia3/SURA/BimodalVideo/hsv_merged/data.yaml')

New https://pypi.org/project/ultralytics/8.2.55 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/data1/jingxia3/SURA/BimodalVideo/hsv_merged/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train43, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

train: Scanning /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/labels.cache... 11375 images, 1047 backgrounds, 0 corrupt: 100%|██████████| 11924/11924 [00:00<?, ?it/s]

train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/images/video-5v4MPkCMGEKPfroxc-frame-000285-M8qDkmPHxWQajYdEK.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/images/video-BjSfmxLQqCGjg8tya-frame-004259-dquYSgZ4rzqYqhtbc.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/train/images/video-JFiGa9oAAN2queuLt-frame-007561-jKiKkJJY9yjxZQjDi.jpg: 1 duplicate labels removed



val: Scanning /data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/labels.cache... 1235 images, 51 backgrounds, 0 corrupt: 100%|██████████| 1235/1235 [00:00<?, ?it/s]

val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-005290-Rkhr9LNNEftZzJ98w.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hsv_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-008818-M29zA3RYL5ud23Cp9.jpg: 5 duplicate labels removed


...

    100/100       8.4G     0.9712      0.545     0.9283         59        640: 100%|██████████| 746/746 [01:28<00:00,  8.47it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [00:04<00:00,  8.90it/s]

                   all       1235      16901      0.816      0.637      0.734      0.463



100 epochs completed in 4.282 hours.
Optimizer stripped from runs/detect/train43/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train43/weights/best.pt, 52.0MB

Validating runs/detect/train43/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8m summary (fused): 218 layers, 25840339 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [00:05<00:00,  6.52it/s]


                   all       1235      16901      0.821      0.636      0.735      0.464
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train43


In [9]:
hot_backbone_m = YOLO("yolov8m.yaml")
hot_res_m = hot_backbone_m.train(data='/data1/jingxia3/SURA/BimodalVideo/hot_merged/data.yaml')

New https://pypi.org/project/ultralytics/8.2.55 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/data1/jingxia3/SURA/BimodalVideo/hot_merged/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train44, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

train: Scanning /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/labels.cache... 11578 images, 1141 backgrounds, 0 corrupt: 100%|██████████| 12127/12127 [00:00<?, ?it/s]

train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/images/video-5v4MPkCMGEKPfroxc-frame-000285-M8qDkmPHxWQajYdEK.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/images/video-BjSfmxLQqCGjg8tya-frame-004259-dquYSgZ4rzqYqhtbc.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/train/images/video-JFiGa9oAAN2queuLt-frame-007561-jKiKkJJY9yjxZQjDi.jpg: 1 duplicate labels removed



val: Scanning /data1/jingxia3/SURA/BimodalVideo/hot_merged/val/labels.cache... 1316 images, 80 backgrounds, 0 corrupt: 100%|██████████| 1316/1316 [00:00<?, ?it/s]

val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-005290-Rkhr9LNNEftZzJ98w.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/hot_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-008818-M29zA3RYL5ud23Cp9.jpg: 5 duplicate labels removed


...

                   all       1316      17073      0.823       0.65      0.743      0.474

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      8.41G          1     0.5655     0.9421        183        640: 100%|██████████| 758/758 [01:29<00:00,  8.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:04<00:00,  8.84it/s]


                   all       1316      17073      0.822       0.65      0.743      0.474

100 epochs completed in 2.655 hours.
Optimizer stripped from runs/detect/train44/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train44/weights/best.pt, 52.0MB

Validating runs/detect/train44/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8m summary (fused): 218 layers, 25840339 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:06<00:00,  6.70it/s]


                   all       1316      17073      0.819      0.652      0.744      0.475
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train44


In [10]:
inferno_backbone_m = YOLO("yolov8m.yaml")
inferno_res_m = inferno_backbone_m.train(data='/data1/jingxia3/SURA/BimodalVideo/inferno_merged/data.yaml')

New https://pypi.org/project/ultralytics/8.2.55 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/data1/jingxia3/SURA/BimodalVideo/inferno_merged/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train45, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

train: Scanning /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/labels.cache... 11631 images, 1159 backgrounds, 0 corrupt: 100%|██████████| 12180/12180 [00:00<?, ?it/s]

train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/images/video-5v4MPkCMGEKPfroxc-frame-000285-M8qDkmPHxWQajYdEK.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/images/video-BjSfmxLQqCGjg8tya-frame-004259-dquYSgZ4rzqYqhtbc.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/train/images/video-JFiGa9oAAN2queuLt-frame-007561-jKiKkJJY9yjxZQjDi.jpg: 1 duplicate labels removed



val: Scanning /data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/labels.cache... 1394 images, 112 backgrounds, 0 corrupt: 100%|██████████| 1394/1394 [00:00<?, ?it/s]

val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-005290-Rkhr9LNNEftZzJ98w.jpg: 3 duplicate labels removed
val: WARNING ⚠️ /data1/jingxia3/SURA/BimodalVideo/inferno_merged/val/images/video-kBvcuDMtYv2Z4kmXi-frame-008818-M29zA3RYL5ud23Cp9.jpg: 5 duplicate labels removed


...

                   all       1394      17223      0.819      0.659      0.748      0.481

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      9.28G     0.9803     0.5523     0.9349         27        640: 100%|██████████| 762/762 [01:30<00:00,  8.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:04<00:00,  9.16it/s]


                   all       1394      17223      0.819      0.661      0.748      0.482

100 epochs completed in 2.699 hours.
Optimizer stripped from runs/detect/train45/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train45/weights/best.pt, 52.0MB

Validating runs/detect/train45/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8m summary (fused): 218 layers, 25840339 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:06<00:00,  6.88it/s]


                   all       1394      17223      0.816      0.662      0.748      0.482
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train45


## Graphs

In [14]:
import numpy as np
import matplotlib.pyplot as plt

# Data for all colormaps
colormaps = ['Original', 'HSV', 'Hot', 'Inferno']
model_sizes = ['Nano', 'Small', 'Medium']
mAP50_data = [
    [0.722, 0.768, 0.793],  # Original
    [0.653, 0.711, 0.735],  # HSV
    [0.665, 0.717, 0.744],  # Hot
    [0.668, 0.725, 0.748]    # Inferno
]
mAP50_95_data = [
    [0.452, 0.498, 0.527],  # Original
    [0.390, 0.439, 0.464],  # HSV
    [0.399, 0.447, 0.475],  # Hot
    [0.401, 0.453, 0.482]   # Inferno
]

x = np.arange(len(model_sizes))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(12, 8))

for i, cmap in enumerate(colormaps):
    mAP50 = np.array(mAP50_data[i], dtype=float)
    mAP50_95 = np.array(mAP50_95_data[i], dtype=float)
    ax.bar(x + (i-1.5) * width, mAP50, width, label=f'{cmap} mAP50')
    ax.bar(x + (i-1.5) * width, mAP50_95, width, label=f'{cmap} mAP50-95', bottom=mAP50)

ax.set_xlabel('Model Size')
ax.set_ylabel('Performance')
ax.set_title('Effect of Colormapping on Training Outcome')
ax.set_xticks(x)
ax.set_xticklabels(model_sizes)
ax.legend()
ax.grid(True)

plt.show()


<Figure size 1200x800 with 1 Axes>

## Benchmarking

### Constructing benchmarking dataset

In [12]:
images_dir = "/data1/jingxia3/SURA/BimodalVideo/rgb_frames_cropped"
labels_dir = "/data1/jingxia3/SURA/BimodalVideo/labels"
output_dir = "/data1/jingxia3/SURA/BimodalVideo/benchmarking_splitted"

create_directory_structure(output_dir)
split_data(images_dir, labels_dir, output_dir)

In [13]:
!cp -rf /data1/jingxia3/SURA/FLIR/final_dataset_rgb /data1/jingxia3/SURA/BimodalVideo/benchmarking

!cp /data1/jingxia3/SURA/BimodalVideo/benchmarking_splitted/train/images/* /data1/jingxia3/SURA/BimodalVideo/benchmarking/train/images/
!cp /data1/jingxia3/SURA/BimodalVideo/benchmarking_splitted/train/labels/* /data1/jingxia3/SURA/BimodalVideo/benchmarking/train/labels/

!cp /data1/jingxia3/SURA/BimodalVideo/benchmarking_splitted/test/images/* /data1/jingxia3/SURA/BimodalVideo/benchmarking/test/images/
!cp /data1/jingxia3/SURA/BimodalVideo/benchmarking_splitted/test/labels/* /data1/jingxia3/SURA/BimodalVideo/benchmarking/test/labels/

!cp /data1/jingxia3/SURA/BimodalVideo/benchmarking_splitted/val/images/* /data1/jingxia3/SURA/BimodalVideo/benchmarking/val/images/
!cp /data1/jingxia3/SURA/BimodalVideo/benchmarking_splitted/val/labels/* /data1/jingxia3/SURA/BimodalVideo/benchmarking/val/labels/

### Benchmarking

In [17]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
metrics = model.val(data='/data1/jingxia3/SURA/BimodalVideo/benchmarking/data.yaml', single_cls=True)

Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients


val: Scanning /data1/jingxia3/SURA/BimodalVideo/benchmarking/val/labels.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1176      17105      0.666      0.347      0.409      0.253
                person       1176      17105      0.666      0.347      0.409      0.253
Speed: 0.7ms preprocess, 1.7ms inference, 0.0ms loss, 4.8ms postprocess per image
Results saved to runs/detect/val83


In [18]:
model = YOLO("yolov8s.pt")
metrics = model.val(data='/data1/jingxia3/SURA/BimodalVideo/benchmarking/data.yaml', single_cls=True)

Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients


val: Scanning /data1/jingxia3/SURA/BimodalVideo/benchmarking/val/labels.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1176      17105      0.705      0.395      0.467      0.299
                person       1176      17105      0.705      0.395      0.467      0.299
Speed: 0.7ms preprocess, 2.6ms inference, 0.0ms loss, 5.9ms postprocess per image
Results saved to runs/detect/val84


In [19]:
model = YOLO("yolov8m.pt")
metrics = model.val(data='/data1/jingxia3/SURA/BimodalVideo/benchmarking/data.yaml', single_cls=True)

Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8m summary (fused): 218 layers, 25886080 parameters, 0 gradients


val: Scanning /data1/jingxia3/SURA/BimodalVideo/benchmarking/val/labels.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1176      17105      0.761       0.45      0.533      0.348
                person       1176      17105      0.761       0.45      0.533      0.348
Speed: 0.6ms preprocess, 5.1ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to runs/detect/val85


### Training with benchmark

In [20]:
benchmark_n = YOLO("yolov8n.yaml")
benchmark_res_n = benchmark_n.train(data='/data1/jingxia3/SURA/BimodalVideo/benchmarking/data.yaml', single_cls=True)

New https://pypi.org/project/ultralytics/8.2.99 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/data1/jingxia3/SURA/BimodalVideo/benchmarking/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train46, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mask

train: Scanning /data1/jingxia3/SURA/BimodalVideo/benchmarking/train/labels... 1


train: New cache created: /data1/jingxia3/SURA/BimodalVideo/benchmarking/train/labels.cache


val: Scanning /data1/jingxia3/SURA/BimodalVideo/benchmarking/val/labels.cache...


Plotting labels to runs/detect/train46/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train46
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      3.42G      1.175     0.6937          1        109        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1176      17105      0.742      0.532      0.606      0.372

100 epochs completed in 4.567 hours.
Optimizer stripped from runs/detect/train46/weights/last.pt, 6.5MB
Optimizer stripped from runs/detect/train46/weights/best.pt, 6.5MB

Validating runs/detect/train46/weights/best.pt...
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1176      17105      0.743      0.531      0.606      0.372
                person       1176      17105      0.743      0.531      0.606      0.372
Speed: 0.2ms preprocess, 0.8ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train46


In [24]:
benchmark_s = YOLO("yolov8s.yaml")
benchmark_res_s = benchmark_s.train(data='/data1/jingxia3/SURA/BimodalVideo/benchmarking/data.yaml', single_cls=True, workers=16)

New https://pypi.org/project/ultralytics/8.2.99 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/data1/jingxia3/SURA/BimodalVideo/benchmarking/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=16, project=None, name=train49, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

train: Scanning /data1/jingxia3/SURA/BimodalVideo/benchmarking/train/labels.cach
val: Scanning /data1/jingxia3/SURA/BimodalVideo/benchmarking/val/labels.cache...


Plotting labels to runs/detect/train49/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 16 dataloader workers
Logging results to runs/detect/train49
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      8.09G      3.844      4.283      3.531        277        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1176      17105      0.193      0.118     0.0743     0.0254

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1176      17105      0.784      0.579      0.661      0.411



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     64/100      8.69G      1.134     0.6713      1.017        185        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1176      17105      0.784      0.581      0.662      0.412

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     65/100       8.5G      1.131     0.6678      1.018        386        640:  IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
benchmark_m = YOLO("yolov8m.yaml")
benchmark_res_m = benchmark_m.train(data='/data1/jingxia3/SURA/BimodalVideo/benchmarking/data.yaml', single_cls=True, workers=16)

New https://pypi.org/project/ultralytics/8.2.99 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.23 🚀 Python-3.12.2 torch-2.3.0 CUDA:0 (NVIDIA A100 80GB PCIe, 81053MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/data1/jingxia3/SURA/BimodalVideo/benchmarking/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=16, project=None, name=train50, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

train: Scanning /data1/jingxia3/SURA/BimodalVideo/benchmarking/train/labels.cach
val: Scanning /data1/jingxia3/SURA/BimodalVideo/benchmarking/val/labels.cache...


Plotting labels to runs/detect/train50/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 16 dataloader workers
Logging results to runs/detect/train50
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      11.4G      3.885      4.442      3.601        277        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       1176      17105      0.209      0.112      0.058     0.0216

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     22/100      11.4G      1.191     0.7224       1.07        144        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all       1176      17105      0.773      0.552      0.629       0.39



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     23/100      12.5G      1.191     0.7219      1.069        426        640:  